In [1]:
import time
import httpx

# Sync vs Async Tests

In [45]:
base_url = "https://swapi.dev/api"
num_iterations = 10

## Fetch URL sync

In [106]:
results = []
s = time.perf_counter()
for i in range(1, 1 + num_iterations):
    url = f"{base_url}/people/{i}/"
    r = httpx.get(url)
    results.append(r.json())
elapsed = time.perf_counter() - s
print(f"fetch executed in {elapsed:0.2f} seconds.")

fetch executed in 2.72 seconds.


## Fetch URL async

In [108]:
# does not work, dunny why
results = []
s = time.perf_counter()
async with httpx.AsyncClient() as client:
    for i in range(1, 1 + num_iterations):
        url = f"{base_url}/people/{i}/"
        r = await client.get(f"{base_url}/people/{i}/")
        results.append(r.json())
elapsed = time.perf_counter() - s
print(f"fetch executed in {elapsed:0.2f} seconds.")

fetch executed in 3.15 seconds.


In [109]:
results = []
urls = [f"{base_url}/people/{i}/" for i in range(1, 1 + num_iterations)]
s = time.perf_counter()
async with httpx.AsyncClient() as client:
    responses = await asyncio.gather(*[client.get(url) for url in urls])
    results = [r.json() for r in responses]
elapsed = time.perf_counter() - s
print(f"fetch executed in {elapsed:0.2f} seconds.")

fetch executed in 1.29 seconds.


## Fetch URL with Threadpool

In [59]:
import concurrent.futures

In [103]:
results = []
urls = [f"{base_url}/people/{i}/" for i in range(1, 1 + num_iterations)]
s = time.perf_counter()
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    future_to_url = {executor.submit(httpx.get, url): url for url in urls}
    for future in concurrent.futures.as_completed(future_to_url):
        url = future_to_url[future]
        try:
            r = future.result()
            results.append(r.json())
        except Exception as exc:
            print('%r generated an exception: %s' % (url, exc))
elapsed = time.perf_counter() - s
print(f"fetch executed in {elapsed:0.2f} seconds.")

fetch executed in 1.32 seconds.


In [62]:
results

[{'name': 'C-3PO',
  'height': '167',
  'mass': '75',
  'hair_color': 'n/a',
  'skin_color': 'gold',
  'eye_color': 'yellow',
  'birth_year': '112BBY',
  'gender': 'n/a',
  'homeworld': 'http://swapi.dev/api/planets/1/',
  'films': ['http://swapi.dev/api/films/1/',
   'http://swapi.dev/api/films/2/',
   'http://swapi.dev/api/films/3/',
   'http://swapi.dev/api/films/4/',
   'http://swapi.dev/api/films/5/',
   'http://swapi.dev/api/films/6/'],
  'species': ['http://swapi.dev/api/species/2/'],
  'vehicles': [],
  'starships': [],
  'created': '2014-12-10T15:10:51.357000Z',
  'edited': '2014-12-20T21:17:50.309000Z',
  'url': 'http://swapi.dev/api/people/2/'},
 {'name': 'Leia Organa',
  'height': '150',
  'mass': '49',
  'hair_color': 'brown',
  'skin_color': 'light',
  'eye_color': 'brown',
  'birth_year': '19BBY',
  'gender': 'female',
  'homeworld': 'http://swapi.dev/api/planets/2/',
  'films': ['http://swapi.dev/api/films/1/',
   'http://swapi.dev/api/films/2/',
   'http://swapi.dev/ap

# Local Webserver Sync View

Sync server started with:
```
gunicorn -b 0.0.0.0:8001 --workers=1 --threads 10 django_async.wsgi
python manage.py runserver 0.0.0.0:8001  # standard dev server is also multithreaded
```

In [2]:
sync_api_url = "http://localhost:8001/sync_api_view/"

### Sync

In [86]:
results = []
s = time.perf_counter()
for i in range(1, 1 + num_iterations):
    r = httpx.get(sync_api_url)
    results.append(r.json())
elapsed = time.perf_counter() - s
print(f"fetch executed in {elapsed:0.2f} seconds.")

fetch executed in 10.18 seconds.


### Async

In [85]:
results = []
s = time.perf_counter()
async with httpx.AsyncClient() as client:
    for i in range(1, 1 + num_iterations):
        r = await client.get(sync_api_url)
        results.append(r.json())
elapsed = time.perf_counter() - s
print(f"fetch executed in {elapsed:0.2f} seconds.")

fetch executed in 10.07 seconds.


In [89]:
results = []
s = time.perf_counter()
for i in range(1, 1 + num_iterations):
    async with httpx.AsyncClient() as client:
        r = await client.get(sync_api_url)
    results.append(r.json())
elapsed = time.perf_counter() - s
print(f"fetch executed in {elapsed:0.2f} seconds.")

fetch executed in 10.20 seconds.


In [96]:
results = []
urls = [sync_api_url for i in range(1, 1 + num_iterations)]
s = time.perf_counter()
async with httpx.AsyncClient() as client:
    responses = await asyncio.gather(*[client.get(url) for url in urls])
    results = [r.json() for r in responses]
elapsed = time.perf_counter() - s
print(f"fetch executed in {elapsed:0.2f} seconds.")

fetch executed in 1.03 seconds.


### Thread

In [88]:
results = []
urls = [sync_api_url for i in range(1, 1 + num_iterations)]
s = time.perf_counter()
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    future_to_url = {executor.submit(httpx.get, url): url for url in urls}
    for future in concurrent.futures.as_completed(future_to_url):
        url = future_to_url[future]
        try:
            r = future.result()
            results.append(r.json())
        except Exception as exc:
            print('%r generated an exception: %s' % (url, exc))
elapsed = time.perf_counter() - s
print(f"fetch executed in {elapsed:0.2f} seconds.")

fetch executed in 1.08 seconds.


In [90]:
from django_async import views

In [91]:
dir(views)

['HttpResponse',
 'JsonResponse',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'async_aggregation_from_external_experiment_view',
 'async_aggregation_from_external_view',
 'async_aggregation_from_sync_view',
 'async_aggregation_view',
 'async_api_view',
 'async_hello_world_view',
 'async_to_sync',
 'asyncio',
 'httpx',
 'sleep',
 'sync_aggregation_view',
 'sync_api_view',
 'sync_external_api_view',
 'sync_hello_world_view',
 'sync_to_async',
 'time']